In [ ]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import math #The Python math module
#import iexfinance
from scipy import stats #The SciPy stats module

In [ ]:
#Get list of all stocks from S&P 500
#https://medium.com/wealthy-bytes/5-lines-of-python-to-automate-getting-the-s-p-500-95a632e5e567
#read_html returns a list of dataframe objects -> in this case two objects where we only need
#the first one. Therefore we select index 0

table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')
df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

stocks = pd.read_csv('S&P500-Info.csv')
#stocks['Symbol']
#stocks = stocks.loc[1:2]
#print(stocks)

In [ ]:
#IEX Parameters 

# Get the relevant Token 
from secrets import IEX_CLOUD_API_TOKEN
token = IEX_CLOUD_API_TOKEN

#from secrets import IEX_SANDBOX_API_TOKEN
#token = IEX_SANDBOX_API_TOKEN

# Get the relevant BASE URL
#Production: https://cloud.iexapis.com/stable
#Sandbox: https://sandbox.iexapis.com/stable
#base_url = 'https://sandbox.iexapis.com/stable'
base_url = 'https://cloud.iexapis.com/stable'


In [ ]:
# Define a final dataframe for further analysis 

my_col = [    'Ticker',                             #1
              'Company Name',                       #2
              'Industry',                           #3
              'Price',                              #4
              'Adjusted 52 week low',               #5
              'Adjusted 52 week high',              #6
              'Market Capitalization',              #7
              'Twelve months Earnings per Share',   #8
              'Price to Earnings Ratio',            #9
              'One-Year Price Return',              #10
              'Six-Month Price Return',             #11
              'Three-Month Price Return',           #12
              'One-Month Price Return',             #13
               ]

fin_df = pd.DataFrame(columns = my_col)
fin_df

In [ ]:
# Define a final dataframe for further analysis 
#resp = requests.get(base_url + '/status')

for ind in stocks.index:
    print(ind, stocks['Symbol'][ind])
    url_company = f"{base_url}/stock/{stocks['Symbol'][ind]}/company?token={token}"   
    data_company = requests.get(url_company).json()
    
    url_quote  = f"{base_url}/stock/{stocks['Symbol'][ind]}/quote?token={token}"
    data_quote = requests.get(url_quote).json()
    
    url_stats  = f"{base_url}/stock/{stocks['Symbol'][ind]}/stats?token={token}"
    data_stats = requests.get(url_stats).json()
    
    #url_advstats  = f"{base_url}/stock/{stocks['Symbol'][ind]}/advanced-stats?token={token}"
    #data_advstats = requests.get(url_advstats).json()
    
    
    fin_df = fin_df.append(pd.Series( [data_quote['symbol'],                   #1
                                       data_quote['companyName'],              #2
                                       data_company['industry'],               #3
                                       data_quote['latestPrice'],              #4
                                       data_quote['week52Low'],                #5
                                       data_quote['week52High'],               #6
                                       data_quote['marketCap'],                #7
                                       data_stats['ttmEPS'],                   #8
                                       data_quote['peRatio'],                  #9
                                       data_stats['year1ChangePercent'],       #10
                                       data_stats['month6ChangePercent'],      #11
                                       data_stats['month3ChangePercent'],      #12 
                                       data_stats['month1ChangePercent'],      #13
                                      ],
                                                    index = my_col), 
                                        ignore_index = True)

fin_df

In [ ]:
#Store raw data extract in to xls.
locpath1 = "C:/Users/marcw/01_projects/jupyterlab/03_algorithmic_trading/"
fin_df.to_excel(locpath1+"fin_df.xlsx", sheet_name='Tabelle1')   

### End of Extraction Programm